# TF-IDF

# Table of Contents

1. [Test Data](#test)
2. [Full Data](#full)
    1. [Clean Data in advance](#clean)

In [1]:
import mwdsbe
import mwdsbe.datasets.licenses as licenses
import schuylkill as skool
import pandas as pd
import numpy as np

In [2]:
registry = mwdsbe.load_registry() # geopandas df
license = licenses.CommercialActivityLicenses().download()

In [3]:
import re

def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

<a id="test"></a>
## Test

In [4]:
# test
mini_registry = registry[:100]

In [5]:
# all_company_names = pd.concat([registry['company_name'].dropna(), license['company_name'].dropna()]).unique()

In [40]:
all_company_names = pd.concat([mini_registry['company_name'].dropna(), license['company_name'].dropna()]).unique()

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(all_company_names)

In [43]:
import time
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 2, 0.85)
t = time.time() - t1

In [44]:
print('Execution time:', t, 'sec')

Execution time: 141.10833144187927 sec


In [45]:
matches_df = get_matches_df(matches, all_company_names, top=1000)

In [46]:
matches

<198383x198383 sparse matrix of type '<class 'numpy.float64'>'
	with 211551 stored elements in Compressed Sparse Row format>

<a id="data"></a>
## Full registry dataset

In [49]:
all_company_names = pd.concat([registry['company_name'].dropna(), license['company_name'].dropna()]).unique()

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(all_company_names)

import time
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 2, 0.85)
t = time.time() - t1

print('Execution time:', t, 'sec')

Execution time: 142.47147250175476 sec


In [188]:
matches_df = get_matches_df(matches, all_company_names, top=100000)

KeyError: 205798

In [91]:
# find the number of real matches only in registry
last_company_name = registry.loc[len(registry) - 1, 'company_name']

In [92]:
last_company_index = matches_df.loc[matches_df['left_side'] == last_company_name].index.tolist()[-1]

In [94]:
real_matches_df = matches_df[:last_company_index + 1]

In [107]:
real_matches = real_matches_df.duplicated(subset='left_side', keep='first')
real_matches = real_matches_df[real_matches]

In [122]:
print('TF-IDF match for raw data: ', len(real_matches), ' out of ', len(registry))

TF-IDF match for raw data:  211  out of  3119


<a id="clean"></a>
### Clean data in advance for full dataset

In [120]:
ignore_words = ['inc', 'group', 'llc', 'corp', 'pc', 'incorporated', 'ltd', 'co']
cleaned_registry = skool.clean_strings(registry, ['company_name', 'dba_name'], True, ignore_words)
cleaned_license = skool.clean_strings(license, ['company_name'], True, ignore_words)

In [199]:
all_company_names = pd.concat([cleaned_registry['company_name'].dropna(), cleaned_license['company_name'].dropna()]).unique()

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(all_company_names)

import time
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 2, 0.85)
t = time.time() - t1

print('Execution time:', t, 'sec')

Execution time: 99.87298631668091 sec


In [200]:
matches_df = get_matches_df(matches, all_company_names, top=100000)
# find the number of real matches only in registry
# last_company_name = registry.loc[len(registry) - 1, 'company_name']

# TODO: need to find the last company name of registry in matches df
last_company_index = 5040

In [193]:
real_matches_df = matches_df[:last_company_index + 1]

# real_matches = real_matches_df.duplicated(subset='left_side', keep='first')
# real_matches = real_matches_df[real_matches]
real_matches = real_matches_df.drop_duplicates(subset='left_side', keep='last', inplace=False)

print('TF-IDF match after cleaning data: ', len(real_matches), ' out of ', len(registry))

TF-IDF match after cleaning data:  4634  out of  3119


In [198]:
real_matches.loc[20]

left_side      a b unique construction home repair
right_side     a b unique construction home repair
similairity                                      1
Name: 20, dtype: object

In [115]:
# To see the difference between registry and tf-idf match
# registry_set = set(registry['company_name'])
# match_set = set(real_matches['left_side'])

In [116]:
# registry_set.difference(match_set)